In [50]:
###  Partially Homomorphic Encryption, multiplicative homomorphic Elgamal, Additive Homomorphic Paillier

import random
from math import pow
import gmpy2 as gy
import time
import libnum

x = 2
y = 3

string = "make american great again"    

byte = bytes(string, 'utf-8')

class Paillier(object):
    
    def __init__(self, pubKey=None, priKey=None):
        self.pubKey = pubKey
        self.priKey = priKey

    def __gen_prime__(self, rs):
        p = gy.mpz_urandomb(rs, 1024)
        while not gy.is_prime(p):
            p += 1
        return p
    
    def __div__(self, x, n):
        res = gy.div((x - 1), n)
        return res
    
    def __key_gen__(self):
        # generate random state
        while True:
            
            rs = gy.random_state(int(time.time()))
            p = self.__gen_prime__(rs)
            q = self.__gen_prime__(rs)
            n = p * q
            lmd =(p - 1) * (q - 1)
          
            if gy.gcd(n, lmd) == 1:
                break
                
        g = n + 1
        mu = gy.invert(lmd, n)
    
        self.pubKey = [n, g]
        self.priKey = [lmd, mu]
        return
        
    def decrypt(self, ciphertext):
        n, g = self.pubKey
        lmd, mu = self.priKey
        m =  self.__div__(gy.powmod(ciphertext, lmd, n ** 2), n) * mu % n
        print("raw message:", m)
        plaintext = libnum.n2s(int(m))
        return plaintext

    def encrypt(self, plaintext):
        m = libnum.s2n(plaintext)
        n, g = self.pubKey
        r = gy.mpz_random(gy.random_state(int(time.time())), n)
        while gy.gcd(n, r)  != 1:
            r += 1
        ciphertext = gy.powmod(g, m, n ** 2) * gy.powmod(r, n, n ** 2) % (n ** 2)
        return ciphertext

if __name__ == "__main__":
    p = Paillier()
    p.__key_gen__()
    Key = p.pubKey
   
    plaintext = byte
    print("Plaintext:", plaintext)
    
    ciphertext = p.encrypt(plaintext)
    print("Ciphertext:", ciphertext)
    
    deciphertext = p.decrypt(ciphertext)
    print("Deciphertext: ", deciphertext)


Plaintext: b'make american great again'
Ciphertext: 689933039060065012734587714597082272483037121300464664877326525811342064288875575900641989012780498270854333209537219475345216691708241441570036536881257621921019872470891344256772419936932436052906033523368526006256283111571186310253414776876113076015717163378987618533009289781426912528153655029410938999954615766556954940904129643552252920763272222451604901008094107495388393462346338239101658710820667131815321719757064205075887008571350734457297820909060307110670151668837886796367138193469276372475070875744272148816363708979043791511636191190605852943635451666632921317871139301943547527805075470890974325135191693450946719195107261539362214044863616881118474837064329870715687854759258632415193869547176759066524255399697538201840382117972260507642364017795581298219695256231381199417083121479834281638629690569523571981837890809071595656360585481648500141290264423312195333150298379301962202240150187789795511311552940485576519274161561802

In [53]:
####  Elgamal    

a = random.randint(2, 10)
 
def gcd(a, b):
    
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b;
    else:
        return gcd(b, a % b)

def gen_key(q):
 
    key = random.randint(pow(10, 20), q)
    while gcd(q, key) != 1:
        key = random.randint(pow(10, 20), q)
 
    return key
 

def power(a, b, c):
    x = 1
    y = a
 
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c;
        y = (y * y) % c
        b = int(b / 2)
 
    return x % c
 

def encrypt(msg, q, h, g):
 
    en_msg = []
 
    k = gen_key(q)
    s = power(h, k, q)
    p = power(g, k, q)
     
    for i in range(0, len(msg)):
        en_msg.append(msg[i])
    
    print("g^k used : ", p)
    print("g^ak used : ", s)
    for i in range(0, len(en_msg)):
        if isinstance(msg, str):
            en_msg[i] = s * ord(en_msg[i])
        else:
            en_msg[i] = s * en_msg[i]
 
    return en_msg, p
 
def decrypt(en_msg, p, key, q):
 
    dr_msg = []
    h = power(p, key, q)
    for i in range(0, len(en_msg)):
        dr_msg.append(chr(int(en_msg[i]/h)))
         
    return dr_msg
 

def main():
 
    plaintext = byte
    print("Plaintext :", plaintext)
 
    q = random.randint(pow(10, 20), pow(10, 50))
    g = random.randint(2, q)
 
    key = gen_key(q)  
    h = power(g, key, q)
    print("g used : ", g)
    print("g^a used : ", h)
 
    ciphertext, p = encrypt(plaintext, q, h, g)
    print("Ciphertext:", ciphertext)
    
    plaintext = decrypt(ciphertext, p, key, q)
    plaintext = ''.join(plaintext)
    print("Deciphertext:", plaintext);
 
 
if __name__ == '__main__':
    main()

Plaintext : b'make american great again'
g used :  32339164616214122668071871182093761165331193744182
g^a used :  78564303451210533221295110342309142971896644443485
g^k used :  90911957763265330905058112622597061037831055789358
g^ak used :  45708931195286426530576306366402348328662402695924
Ciphertext: [4982273500286220491832817393937855967824201893855716, 4433766325942783373465901717541027787880253061504628, 4890855637895647638771664781205051271166877088463868, 4616602050723929079588206943006637181194902672288324, 1462685798249165648978441803724875146517196886269568, 4433766325942783373465901717541027787880253061504628, 4982273500286220491832817393937855967824201893855716, 4616602050723929079588206943006637181194902672288324, 5210818156262652624485698925769867709467513907335336, 4799437775505074785710512168472246574509552283072020, 4525184188333356226527054330273832484537577866896476, 4433766325942783373465901717541027787880253061504628, 50279824314815069183633937003042583161528642965